## Song Review Project

**Author:** Vanessa Ma <br>
**Last Modified:** 16/12/2022 <br>

This project will transform two csv files. One contains song reviews from a group of users, and the other contains the song names that were put in by users. Sentiment Analysis from Azure Cognitive Services is then used to determine the overall sentiment of users and who had the most popular song selection

Future Ideas - use Spotify and Apple Music APIs to pull a playlist and identify users automatically

### Import Libraries and Variables

In [122]:
#data transformations and visualisation
import pandas as pd
import seaborn as sns
import numpy as np

#azure services
from azure.ai.textanalytics import TextAnalyticsClient
from azure.core.credentials import AzureKeyCredential

#so I can have dotenv variables
from dotenv import load_dotenv
import os

In [123]:
#Get azure variables
load_dotenv()

key = os.environ['KEY']
endpoint = os.environ['ENDPOINT']
location = os.environ['LOCATION']

In [124]:
credential = AzureKeyCredential(key)
text_analytics_client = TextAnalyticsClient(endpoint=endpoint, credential=credential)

### Data Transformation

In [125]:
#'utf-8' codec can't decode byte 0xa0 in position 16: invalid start byte when encoding not specified
song_reviews = pd.read_csv('song-review-data/Song Data.csv', encoding='unicode_escape')
song_selectors = pd.read_csv('song-review-data/whodata.csv', encoding='unicode_escape')

In [126]:
display(song_reviews) #can see that the quotation mark isn't being read properly 

,Unnamed: 0,Mikaela,Vanessa,Giles
0,Monster,Halloween vibes and finger snaps are what get ...,"I get dark seductress vibes, very red velvet p...",Chorus makes me laugh
1,Lemonworld,Little bro got me into Aussie bands and now I...,Definitely a vibe. Ocean Alley is what Id pla...,chill aussie slacker rock
2,Denim Jacket,Funky vibes. A bop. Love it.,NaN,A very 70s funk sound
3,Live In Life,"Kinda wanna die, and I kinda wanna live in life",Nice and chill,Not usually a fan of the Rubens but this ones...
4,Robbery,"New to Aussie bands so this is a funky, fresh ...","Yessss, is the type of song Im listening to now",Pretty similar to Lime Cordiales other songs ...
...,...,...,...,...
69,There Is a Light That Never Goes Out,Love almost any song that has a strong guitar ...,"Okay, by this point the instrumental is gettin...",Dreamy and melancholy
70,Walking On The Moon,This police song is good but not as good as RO...,Police is goood. Sting is too good,I would walk on the moon to this
71,NaN,Reggae vibes? No? Yes? I dont know music genres.,NaN,NaN
72,Appointment,I doubt this will shock anyone but Im not a b...,Im not a fan of the instruments that come in ...,"Bittersweet slowburner, a real ballad"


In [127]:
display(song_selectors.head(10))

,Mikaela,Vanessa,Giles
0,Monster,NaN,NaN
1,Lemonworld,NaN,NaN
2,Live In Life,NaN,NaN
3,Robbery,NaN,NaN
4,Temper Temper,NaN,NaN
5,NaN,Denim Jacket,NaN
6,NaN,Heavyweight Champion of the World,NaN
7,Cant You See Me?,NaN,NaN
8,NaN,Lottery,NaN
9,NaN,NaN,NaN


After a view of the data, we want to ensure we know whose song is associated to the right person. The first possible approach, given the way the data was structured, is _some sort of index join_ but instead of the song name we put our name. The second possible approach is a sort of _for loop where we match the song title, and then put the name in_ but that could be clunky. We shall see...

**Aproach 1**

In [128]:
#for all non_NaN columns, replace with the column name

for col in song_selectors.columns:
    print(f'Processing {col}')
    
    song_selectors[col][song_selectors[col].notnull()] = col
    song_selectors[col][song_selectors[col].isnull()] = ''
    
    print('\n')

Processing Mikaela


Processing Vanessa


Processing Giles




In [129]:
song_selectors

,Mikaela,Vanessa,Giles
0,Mikaela,,
1,Mikaela,,
2,Mikaela,,
3,Mikaela,,
4,Mikaela,,
...,...,...,...
69,,,Giles
70,,,Giles
71,,,
72,,,Giles


In [130]:
#merge the columns
song_selectors['selector'] = song_selectors['Mikaela'].astype(str) + song_selectors['Vanessa'].astype(str) + song_selectors['Giles'].astype(str)
song_selectors

,Mikaela,Vanessa,Giles,selector
0,Mikaela,,,Mikaela
1,Mikaela,,,Mikaela
2,Mikaela,,,Mikaela
3,Mikaela,,,Mikaela
4,Mikaela,,,Mikaela
...,...,...,...,...
69,,,Giles,Giles
70,,,Giles,Giles
71,,,,
72,,,Giles,Giles


In [131]:
song_selectors['selector']

0     Mikaela
1     Mikaela
2     Mikaela
3     Mikaela
4     Mikaela
       ...   
69      Giles
70      Giles
71           
72      Giles
73    Vanessa
Name: selector, Length: 74, dtype: object

In [132]:
#lol, much easier to just add new column
song_reviews['selector'] = song_selectors['selector']

In [133]:
song_reviews

,Unnamed: 0,Mikaela,Vanessa,Giles,selector
0,Monster,Halloween vibes and finger snaps are what get ...,"I get dark seductress vibes, very red velvet p...",Chorus makes me laugh,Mikaela
1,Lemonworld,Little bro got me into Aussie bands and now I...,Definitely a vibe. Ocean Alley is what Id pla...,chill aussie slacker rock,Mikaela
2,Denim Jacket,Funky vibes. A bop. Love it.,NaN,A very 70s funk sound,Mikaela
3,Live In Life,"Kinda wanna die, and I kinda wanna live in life",Nice and chill,Not usually a fan of the Rubens but this ones...,Mikaela
4,Robbery,"New to Aussie bands so this is a funky, fresh ...","Yessss, is the type of song Im listening to now",Pretty similar to Lime Cordiales other songs ...,Mikaela
...,...,...,...,...,...
69,There Is a Light That Never Goes Out,Love almost any song that has a strong guitar ...,"Okay, by this point the instrumental is gettin...",Dreamy and melancholy,Giles
70,Walking On The Moon,This police song is good but not as good as RO...,Police is goood. Sting is too good,I would walk on the moon to this,Giles
71,NaN,Reggae vibes? No? Yes? I dont know music genres.,NaN,NaN,
72,Appointment,I doubt this will shock anyone but Im not a b...,Im not a fan of the instruments that come in ...,"Bittersweet slowburner, a real ballad",Giles


### Selecting the relevant data

In [178]:
test = song_reviews['Vanessa'].loc[song_reviews['selector'] == 'Vanessa']
test = test.loc[test.notnull()].to_list()

In [153]:
test = test.head(15).loc[test.notnull()].to_list()

### Testing how the Text Analytics Client works

**Input** <br>
Each document can be passed as a list of strings, but you can also pass it through with a dict-like representation
<br>
limit of 10

In [154]:
response = text_analytics_client.analyze_sentiment(test)
successful_responses = [doc for doc in response if not doc.is_error]

In [156]:
print(successful_responses[0]['confidence_scores'])
print('\n')
print(successful_responses[0]['sentences'])

{'positive': 1.0, 'neutral': 0.0, 'negative': 0.0}


[SentenceSentiment(text=Good beat, groovy, I would have this on my playlist, sentiment=positive, confidence_scores=SentimentConfidenceScores(positive=1.0, neutral=0.0, negative=0.0), length=51, offset=0, mined_opinions=[])]


In [157]:
print(successful_responses[1]['confidence_scores'])
print('\n')
print(successful_responses[1]['sentences'])

{'positive': 0.01, 'neutral': 0.98, 'negative': 0.0}


[SentenceSentiment(text=The harmony around 2:10-3:00, sentiment=neutral, confidence_scores=SentimentConfidenceScores(positive=0.01, neutral=0.98, negative=0.0), length=28, offset=0, mined_opinions=[])]


In [158]:
print(successful_responses[2]['confidence_scores'])
print('\n')
print(successful_responses[2]['sentences'])

{'positive': 0.31, 'neutral': 0.46, 'negative': 0.23}


[SentenceSentiment(text=I live for the transition between suga and rm and j-hopes rap but omg, just the whole song is fire, sentiment=neutral, confidence_scores=SentimentConfidenceScores(positive=0.31, neutral=0.46, negative=0.23), length=99, offset=0, mined_opinions=[])]


### Problem encountered
How are we going to determine the true score?

We've got positive, neutral and negative. We could go for some sort of mathematical formula... or we could combine it into one massive sentence?

In [179]:
test_join = '. '.join(test)
display(test_join)

'Good beat, groovy, I would have this on my playlist. The harmony around 2:10-3:00. I live for the transition between suga and rm and j-hope\x92s rap but omg, just the whole song is fire. Super musical-like, just imagine the glitzy production that would go along. This is the sound of love without being overly cheesy. Listen to that BEAT. Her emotion is feels. Pretend you\x92ve been transported to a mythical land and the girl is walking through a snowy street with orange lamps softly glowing. And then she wisps away to space where she\x92s floating amongst the stars. The normal world then appears as she sees someone she loves and is gently caressing their face. I just love the reference to Beyonce\x92s halo and it\x92s great to sing. I love the song\x92s theme. Eeevil Womannn. 0:53-1:10 is my fav. Covid vibes. Production is very slick, Pharrell Williams has a slick voice, weirdly metaphorical lyrics. I don\x92t know if it\x92s the electric guitar or the bass guitar, but it vibes. I like

In [180]:
test_joined = []
test_joined.append(test_join)

In [181]:
response = text_analytics_client.analyze_sentiment(test_joined)
successful_responses = [doc for doc in response if not doc.is_error]

In [182]:
successful_responses[0]['confidence_scores']

SentimentConfidenceScores(positive=0.72, neutral=0.05, negative=0.23)